# Electricity Demand Prediction

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os
import glob
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Merging the data

In [ ]:

# Specify the folder path
folder_path = 'D:\\My desktop\\Desktop\\Yearly Merged Files'  # Replace with your folder path

# Get all .xlsx files in the folder
xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
allData = []
# Loop through each file
for file_path in xlsx_files:
    print(f"\nProcessing file: {os.path.basename(file_path)}")
    
    # Read the Excel file
    df = pd.read_excel(file_path)
    df = df.iloc[:-4]
    print(f"Initial DataFrame shape: {df.shape}")
    allData.append(df)
    # Process your data here
    # print(f"Columns: {df.columns.tolist()}")
    # print(f"Last 5 rows:\n{df.tail()}")

merged_df = pd.concat(allData, ignore_index=True)   
print(merged_df.tail())


In [ ]:
merged_df["Year"]=[merged_df["State"][i].split("-")[-1].strip() for i in range(len(merged_df["State"]))]
merged_df["State"]=[merged_df["State"][i].split("-")[0].strip() for i in range(len(merged_df["State"]))]
# merged_df = merged_df[merged_df['State'] == 'West Bengal']


In [ ]:

print(f"\nMerged DataFrame shape: {merged_df.shape}")
merged_df.to_csv("D:\\My desktop\\Desktop\\Yearly Merged Files\\Merged_Data_17_24.csv", index=False)
print("Data merged and saved successfully.")

In [ ]:
merged_df=pd.read_csv("D:\\My desktop\\Desktop\\Yearly Merged Files\\Merged_Data_17_24.csv")
merged_df["Date"],merged_df['Time']=[merged_df['Date'][i].split(" ")[0]+"-"+str(merged_df['Year'][i]) for i in range(len(merged_df["Date"]))],[merged_df['Date'][i].split(" ")[1] for i in range(len(merged_df["Date"]))]

In [ ]:
merged_df.head()

In [ ]:
month={ 'Apr':'04',
        'Aug': '08',
        'Dec': '12',
        'Feb': '02',
        'Jan': '01',
        'Jul': '07',
        'Jun': '06',
        'Mar': '03',
        'May': '05',
        'Nov': '11',
        'Oct': '10',
        'Sep': '09' }
merged_df['Date'] = merged_df['Date'].replace(month, regex=True)

In [ ]:
merged_df.head()

In [ ]:
merged_df['Time'] = pd.to_datetime(merged_df['Time'], format='%I%p').dt.time

In [ ]:
merged_df["DateTime"] = pd.to_datetime(merged_df['Date'].astype(str) + ' ' + merged_df['Time'].astype(str), format='%d-%m-%Y %H:%M:%S')

In [ ]:
merged_df.head()

In [ ]:
merged_df = merged_df.drop(['Date', 'Year','Time'], axis=1)

In [ ]:
west_bengal_data = merged_df[merged_df['State']=='West Bengal']

In [ ]:
west_bengal_data.shape

In [ ]:
west_bengal_data.info()

In [ ]:
west_bengal_data['DateTime'] = pd.to_datetime(west_bengal_data['DateTime'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
west_bengal_data.sample(5)

In [ ]:
west_bengal_data.to_csv("D:\\My desktop\\Desktop\\Yearly Merged Files\\West_Bengal_Data_17_24.csv")

## Applying MSTL model

In [ ]:
from statsmodels.tsa.seasonal import MSTL
# Apply MSTL decomposition
mstl = MSTL(west_bengal_data["Hourly Demand Met (in MW)"], periods=(24,24*365))  # seasonal=13 for monthly data
result_mstl = mstl.fit()

In [ ]:
# Extract components
west_bengal_data['trend'] = result_mstl.trend
west_bengal_data['daily_seasonal'] = result_mstl.seasonal['seasonal_24']
west_bengal_data['yearly_seasonal'] = result_mstl.seasonal['seasonal_8760']  # 8760 = 24*365
west_bengal_data['resid'] = result_mstl.resid

In [ ]:

from plotly.subplots import make_subplots
# Create subplots
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=("Trend", "Daily Seasonality","Yearly Seasonality", "Residuals")
)

# Add traces for each component
fig.add_trace(
    go.Scatter(x=west_bengal_data['DateTime'], y=west_bengal_data['trend'], name='Trend'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=west_bengal_data['DateTime'], y=west_bengal_data['daily_seasonal'], name='Daily'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=west_bengal_data['DateTime'], y=west_bengal_data['yearly_seasonal'], name='Yearly'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=west_bengal_data['DateTime'], y=west_bengal_data['resid'], name='Residual'),
    row=4, col=1
)


# Update layout
fig.update_layout(
    title_text="MSTL Decomposition west_bengal_data",
    height=1200,
    showlegend=False,
    template="plotly_dark",  # Optional: for dark mode
    xaxis4=dict(
        rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
        type="date"
    )
)

# Update y-axis titles
fig.update_yaxes(title_text="Trend", row=1, col=1)
fig.update_yaxes(title_text="Daily", row=2, col=1)
fig.update_yaxes(title_text="Yearly", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

fig.show()

In [ ]:
west_bengal_data.to_csv("D:\\My desktop\\Desktop\\Yearly Merged Files\\West_Bengal_Data_17_24_MSTL.csv")

In [ ]:
west_bengal_data = pd.read_csv("D:\\My desktop\\Desktop\\Yearly Merged Files\\West_Bengal_Data_17_24_MSTL.csv")

### Estimating the trend Component

In [ ]:
from patsy import dmatrix
import statsmodels.api as sm
x=np.arange(len(west_bengal_data['trend'][:-24*486]))
spline_basis = dmatrix("bs(x, df=200, degree=3, include_intercept=False)", {"x": x}, return_type='dataframe')
y= west_bengal_data['trend'][:-24*486].values
ols_model = sm.OLS(y, spline_basis).fit()
x_final=np.arange(len(west_bengal_data['trend']))
X_final = dmatrix("bs(x, df=200, degree=3)", {"x": x_final})
predicted_trend = ols_model.predict(X_final)
y_final = west_bengal_data['trend'].values  


plt.plot(west_bengal_data['DateTime'],y_final, label='Original')
plt.plot(west_bengal_data['DateTime'],predicted_trend, label='Spline Fit', color='green')
plt.xlabel("DateTime")
plt.ylabel("Trend")
plt.legend()
plt.title("Spline Trend Fit")
plt.show()

In [ ]:
mse_trend_spline = mean_squared_error(west_bengal_data['trend'][-24*486:], predicted_trend[-24*486:])
mae_trend_spline = mean_absolute_error(west_bengal_data['trend'][-24*486:], predicted_trend[-24*486:])
mse_table_trend = pd.DataFrame({
    'Model': ['Spline'],
    'MSE(Trend)': [f"{mse_trend_spline:.2f}"],
    'MAE(Trend)': [f"{mae_trend_spline:.2f}"]
})
print(mse_table_trend)

### Estimating Daily Seasonality

##### Using Dummy variables

We will create 24 dummie variables consisting of 24 hours. Then we will try to fit a Linear Model to predict the daily seasonality.

In [ ]:
hour_dummies = pd.get_dummies(west_bengal_data['DateTime'].dt.time, prefix='hour', drop_first=True)

In [ ]:
hour_dummies

In [ ]:
from sklearn.linear_model import LinearRegression
# Fit a linear regression model to predict seasonality from hour dummies
lr_model = LinearRegression()
lr_model.fit(hour_dummies[:-24*486], west_bengal_data['daily_seasonal'][:-24*486])

# Predict seasonality from hour
daily_seasonality= lr_model.predict(hour_dummies)

In [ ]:
forecast_steps = 24 * 486 # Forecast for the next 7 days (24 hours each)
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['daily_seasonal'][-forecast_steps:], name='MSTL Daily Seasonality'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=daily_seasonality[-forecast_steps:], name='Forecast_Dummy',opacity=0.6))
fig.update_layout(
    title='Daily Seasonality using Dummy Variable vs MSTL daily seasonality',
    xaxis_title='DateTime',
    yaxis_title='Daily Seasonal Component',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()


In [ ]:
lr_model.coef_

In [ ]:
forecast_steps=24*486
actual = west_bengal_data['daily_seasonal'][-forecast_steps:]
forecast = daily_seasonality[-forecast_steps:]
# Calculate MSE
mse_Dummy_daily = mean_squared_error(actual, forecast)
mae_Dummy_daily = mean_absolute_error(actual, forecast)
print(f"MSE for the lr model to predict daily seasonality using dummy variables  : {mse_Dummy_daily:.2f}")
print(f"MAE for the lr model to predict daily seasonality using dummy variables  : {mae_Dummy_daily:.2f}")

### Using TBATS

In [ ]:
from tbats import TBATS


model_daily = TBATS(
    seasonal_periods=[24],
  
    use_box_cox=False,
    
    use_arma_errors=True
)

In [ ]:
fitted_model_daily = model_daily.fit( west_bengal_data['daily_seasonal'][:-24*486])

In [ ]:
# Forecast trend + residual
forecast_steps = 24 * 486  # 486 days ahead
daily_seasonality_tbats = fitted_model_daily.forecast(steps=forecast_steps)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['daily_seasonal'][-forecast_steps:], name='MSTL Daily Seasonality'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=daily_seasonality_tbats, name='Forecast_TBATS',opacity=0.6))
fig.update_layout(
    title='TBATS Forecast vs MSTL Daily Seasonality',
    xaxis_title='DateTime',
    yaxis_title='Daily Seasonal Component',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()

In [ ]:
import pandas as pd

def get_tbats_parameters(model):
    # These attributes are usually always present
    alpha = getattr(model.params, 'alpha', "N/A")
    
    # These might be None or missing depending on AIC selection
    beta = getattr(model.params, 'beta', "Not Used")
    phi = getattr(model.params, 'phi', "Not Used")
    
    # Since you set use_box_cox=False, lambda is effectively 1.0
    box_cox = "1.0 (Disabled)"
    
    # ARMA parameters - these will exist if AIC selected p,q > 0
    ar_params = getattr(model.params, 'ar_params', "Not Used")
    ma_params = getattr(model.params, 'ma_params', "Not Used")
    
    # Seasonal parameters for your 24-period cycle
    gamma = getattr(model.params, 'gamma_params', "Not Used")
    
    return {
        "Level (Alpha)": alpha,
        "Trend (Beta)": beta,
        "Damping (Phi)": phi,
        "Box-Cox (Lambda)": box_cox,
        "AR Coeffs": ar_params,
        "MA Coeffs": ma_params,
        "Seasonal (Gamma)": list(gamma) if isinstance(gamma, (np.ndarray, list)) else gamma
    }

# Create the table
df_params = pd.DataFrame([get_tbats_parameters(fitted_model_daily)]).T
df_params.columns = ['Value']
print(df_params)

In [ ]:
mse_tbats_daily = mean_squared_error(actual, daily_seasonality_tbats)
mae_tbats_daily = mean_absolute_error(actual, daily_seasonality_tbats)
print(f"MSE for the lr model to predict daily seasonality using tbats  : {mse_tbats_daily:.2f}")
print(f"MAE for the lr model to predict daily seasonality using tbats  : {mae_tbats_daily:.2f}")

In [ ]:
mse_tbats_daily = mean_squared_error(west_bengal_data['daily_seasonal'][-forecast_steps:], daily_seasonality_tbats)
mae_tbats_daily = mean_absolute_error(west_bengal_data['daily_seasonal'][-forecast_steps:], daily_seasonality_tbats)
mse_Dummy_daily = mean_squared_error(west_bengal_data['daily_seasonal'][-forecast_steps:], daily_seasonality[-forecast_steps:])
mae_Dummy_daily = mean_absolute_error(west_bengal_data['daily_seasonal'][-forecast_steps:], daily_seasonality[-forecast_steps:])
mse_table_daily = pd.DataFrame({
    'Model': ['Dummy', 'TBATS'],
    'MSE(Daily)': [f"{mse_Dummy_daily:.2f}", f"{mse_tbats_daily:.2f}"],
    'MAE(Daily)': [f"{mae_Dummy_daily:.2f}", f"{mae_tbats_daily:.2f}"]
})
print(mse_table_daily)

So, using dummy variable explains better than tbats in forecasting daily seasonality.

### Modelling Yearly Seasonality

##### Using Dummy Variables

In [ ]:
month_dummies = pd.get_dummies(west_bengal_data['DateTime'].dt.month, prefix='month', drop_first=True)

In [ ]:
month_dummies

In [ ]:
lr_model_monthly = LinearRegression()
lr_model_monthly.fit(month_dummies[:-24*486], west_bengal_data['yearly_seasonal'][:-24*486])

# Predict seasonality from hour
monthly_seasonality= lr_model_monthly.predict(month_dummies)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['yearly_seasonal'][-forecast_steps:], name='MSTL Yearly Seasonality'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=monthly_seasonality, name='Forecast_Dummy'))
fig.update_layout(
    title='Yearly Seasonality Using Dummy vs MSTL Yearly Seasonality',
    xaxis_title='DateTime',
    yaxis_title='Yearly Seasonal Component',
    xaxis=dict(
        rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
        type="date"  # Use date scale for x-axis
    ),
    legend=dict(
          # Horizontal legend
        yanchor="top",
        y=1.2,  # Position below plot
        xanchor="center",
        x=0.9,
        borderwidth=1,
        bordercolor="Black",
        
     
    ),
    template='plotly_white'
)
fig.show()

### Using TBATS

In [ ]:
from tbats import TBATS

# Fit TBATS on deseasonalized data
model = TBATS(
    seasonal_periods=[24*365],  # TBATS handles seasonality internally
  
    use_box_cox=False,
    
    use_arma_errors=True
)

In [ ]:
fitted_model_yearly=model.fit(west_bengal_data['yearly_seasonal'][:-24*486])

In [ ]:
yearly_seasonality_tbats = fitted_model_yearly.forecast(steps=forecast_steps)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['yearly_seasonal'][-forecast_steps:], name='MSTL Yearly Seasonality'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=yearly_seasonality_tbats, name='Forecast_TBATS'))
fig.update_layout(
    title='Yearly Seasonality Using Dummy vs MSTL Yearly Seasonality',
    xaxis_title='DateTime',
    yaxis_title='Yearly Seasonal Component',
    xaxis=dict(
        rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
        type="date"  # Use date scale for x-axis
    ),
    legend=dict(
          # Horizontal legend
        yanchor="top",
        y=1.2,  # Position below plot
        xanchor="center",
        x=0.9,
        borderwidth=1,
        bordercolor="Black",
        
     
    ),
    template='plotly_white'
)
fig.show()

In [ ]:
mse_tbats_yearly = mean_squared_error(west_bengal_data['yearly_seasonal'][-forecast_steps:], yearly_seasonality_tbats)
mae_tbats_yearly = mean_absolute_error(west_bengal_data['yearly_seasonal'][-forecast_steps:], yearly_seasonality_tbats)
mse_Dummy_yearly = mean_squared_error(west_bengal_data['yearly_seasonal'][-forecast_steps:], monthly_seasonality[-forecast_steps:])
mae_Dummy_yearly = mean_absolute_error(west_bengal_data['yearly_seasonal'][-forecast_steps:], monthly_seasonality[-forecast_steps:])
mse_table_yearly = pd.DataFrame({
    'Model': ['Dummy', 'TBATS'],
    'MSE(Yearly)': [f"{mse_Dummy_yearly:.2f}", f"{mse_tbats_yearly:.2f}"],
    'MAE(Yearly)': [f"{mae_Dummy_yearly:.2f}", f"{mae_tbats_yearly:.2f}"]
})
print(mse_table_yearly)

## Checking the Residuals

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(west_bengal_data['resid'], autolag='AIC')
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Used lags:", result[2])
print("Number of observations:", result[3])
print("Critical Values:")
for k, v in result[4].items():
    print(f"  {k}%: {v}")

Hence the series is stationary.

In [ ]:

from statsmodels.tsa.stattools import acf,pacf
import numpy as np

# Example: ACF of STL residuals (replace `residuals` with your data)
residuals = west_bengal_data['resid']  # From previous STL decomposition

# Compute ACF and confidence intervals
acf_values = acf(residuals, nlags=600, fft=True)  # nlags = max lag to compute
conf_int = 1.96 / np.sqrt(len(residuals))  # 95% confidence interval

# Create Plotly figure
fig = go.Figure()

# Add ACF bars
fig.add_trace(
    go.Bar(
        x=list(range(len(acf_values))),
        y=acf_values,
        name="ACF",
    )
)

# Add confidence interval lines
fig.add_hline(
    y=conf_int,
    line_dash="dot",
    line_color="red",
    annotation_text="95% CI",
    annotation_position="bottom right"
)
fig.add_hline(
    y=-conf_int,
    line_dash="dot",
    line_color="red"
)

# Customize layout
fig.update_layout(
    title="Autocorrelation (ACF) Plot",
    xaxis_title="Lag",
    yaxis_title="Correlation",
    showlegend=False,
    template="plotly_white",
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="linear"  # Use linear scale for x-axis
    # )
)

fig.show()

In [ ]:
# 1. Compute PACF values (nlags = number of lags to compute)
pacf_values = pacf(west_bengal_data['resid'], nlags=400)

# 2. Plot using Plotly
lags = list(range(len(pacf_values)))

fig = go.Figure()

# Bars
fig.add_trace(go.Bar(x=lags, y=pacf_values, name='PACF'))

# Confidence intervals (approx ±1.96/√n)
import numpy as np
conf_level = 1.96 / np.sqrt(len(west_bengal_data['resid']))

fig.add_shape(type="line", x0=0, x1=len(lags), y0=conf_level, y1=conf_level,
              line=dict(color="red", dash="dash"))
fig.add_shape(type="line", x0=0, x1=len(lags), y0=-conf_level, y1=-conf_level,
              line=dict(color="red", dash="dash"))

# Layout
fig.update_layout(title="Partial Autocorrelation Function (PACF)",
                  xaxis_title="Lag",
                  yaxis_title="PACF",
                  showlegend=False)
fig.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Perform Ljung-Box test (null hypothesis: no autocorrelation)
ljung_box_result = acorr_ljungbox(west_bengal_data['resid'], lags=[1,24], return_df=True)
print(ljung_box_result)

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw)


So, there is a significant autocorrelation.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(west_bengal_data['resid'][:-24*486], order=(1, 0, 1))  # Example order
model_fit = model.fit()

In [ ]:
print("AIC: ",model_fit.aic)

In [ ]:
out_sample_forecast = model_fit.forecast(steps=len(west_bengal_data['DateTime'][-24*486:]))


In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['resid'][-forecast_steps:], name='Observed'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=out_sample_forecast, name='Forecast'))
fig.update_layout(
    title='Forecast vs Observed',
    xaxis_title='DateTime',
    yaxis_title='Residuals',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()

### Final Forecast

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
forecast=predicted_trend[-24*486:]+daily_seasonality[-24*486:]+monthly_seasonality[-24*486:]
actual=west_bengal_data['Hourly Demand Met (in MW)'][-24*486:]
mse=mean_squared_error(actual,forecast)
mae=mean_absolute_error(actual,forecast)
print(f"MSE for the final forecast: {mse:.2f}\nMAE for the final forecast: {mae:.2f}")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
forecast=predicted_trend[-24*486:]+daily_seasonality_tbats+yearly_seasonality_tbats
actual=west_bengal_data['Hourly Demand Met (in MW)'][-24*486:]
mse=mean_squared_error(actual,forecast)
mae=mean_absolute_error(actual,forecast)
print(f"Using TBATS\nMSE for the final forecast: {mse:.2f}\nMAE for the final forecast: {mae:.2f}")

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=actual, name='Observed'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=forecast, name='Forecast',mode='lines',opacity=0.6))  # 60% opaque (40% transparent)
fig.update_layout(
    title='Forecast vs Observed',
    xaxis_title='DateTime',
    yaxis_title='Hourly Demand Met (in MW)',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()

## Another approach

At first we will fit a linear trend to the whole data.

In [ ]:
### Fitting a linear trend
# 1. Convert 'DateTime' to datetime type
west_bengal_data['DateTime'] = pd.to_datetime(west_bengal_data['DateTime'])
west_bengal_data['time_index'] = (west_bengal_data['DateTime'] - west_bengal_data['DateTime'].min()).dt.total_seconds() / 3600  # time in hours
from sklearn.linear_model import LinearRegression

# 3. Fit linear trend
X = west_bengal_data[['time_index']][:-24*486]
y = west_bengal_data['Hourly Demand Met (in MW)'][:-24*486]
model = LinearRegression()
model.fit(X, y)
west_bengal_data['linear_trend'] = model.predict(west_bengal_data[['time_index']])

# 4. Plot
plt.figure(figsize=(12, 6))
plt.plot(west_bengal_data['DateTime'], west_bengal_data['Hourly Demand Met (in MW)'], label='Original Data')
plt.plot(west_bengal_data['DateTime'], west_bengal_data['linear_trend'], color='red', label='Linear Trend')
plt.xlabel("DateTime")
plt.ylabel("Value")
plt.title("Fitted Linear Trend")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print("Linear Trend Coefficient:", model.coef_[0])
print("Linear Trend Intercept:", model.intercept_)

In [ ]:
west_bengal_data['detrended']=west_bengal_data['Hourly Demand Met (in MW)'] - west_bengal_data['linear_trend']

In [ ]:
### Daily seasonality
hour_dummies = pd.get_dummies(west_bengal_data['DateTime'].dt.time, prefix='hour', drop_first=True)
from sklearn.linear_model import LinearRegression
# Fit a linear regression model to predict seasonality from hour dummies
lr_model = LinearRegression()
lr_model.fit(hour_dummies[:-24*486], west_bengal_data['detrended'][:-24*486])

# Predict seasonality from hour
daily_seasonality= lr_model.predict(hour_dummies)

In [ ]:
print('Intercept: ',lr_model.intercept_,"Coefficients: ",lr_model.coef_)

In [ ]:
forecast_steps = 24 * 486  # Forecast for the next 486 days (24 hours each)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['detrended'][-forecast_steps:], name='Observed'))
fig.add_trace(go.Scatter(
    x=west_bengal_data['DateTime'][-forecast_steps:],
    y=daily_seasonality[-forecast_steps:], 
    name='Forecast_Dummy',
    mode='lines',
    opacity= 0.6  # 60% opaque (40% transparent)
        ))

fig.update_layout(
    title='Seasonality Forecast vs Observed',
    xaxis_title='DateTime',
    yaxis_title='Daily Seasonal Component',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    legend=dict(
          # Horizontal legend
        yanchor="top",
        y=1,  # Position below plot
        xanchor="center",
        x=0.9,
        borderwidth=1,
        bordercolor="Black",
        
     
    ),
    width=810,
    
    template='plotly_white'
)
fig.show()

In [ ]:
west_bengal_data['detrended_daily_seasonality'] = west_bengal_data['detrended'] - daily_seasonality

In [ ]:
month_dummies = pd.get_dummies(west_bengal_data['DateTime'].dt.month, prefix='month', drop_first=True)
lr_model_monthly = LinearRegression()
lr_model_monthly.fit(month_dummies[:-24*486], west_bengal_data['detrended_daily_seasonality'][:-24*486])

# Predict seasonality from hour
monthly_seasonality= lr_model_monthly.predict(month_dummies)

In [ ]:
print('Model Coefficients : ',lr_model_monthly.coef_,'\nModel Intercept : ',lr_model_monthly.intercept_)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['detrended_daily_seasonality'][-forecast_steps:], name='Detrended Daily Seasonality'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=monthly_seasonality[-forecast_steps:], name='Forecast_Dummy'))
fig.update_layout(
    title='Seasonality Forecast vs Detrended Daily Seasonality',
    xaxis_title='DateTime',
    yaxis_title='Yearly Seasonal Component',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    legend=dict(
          # Horizontal legend
        yanchor="top",
        y=1,  # Position below plot
        xanchor="center",
        x=0.9,
        borderwidth=1,
        bordercolor="Black",
        
     
    ),
    width=810,
    template='plotly_white'
)
fig.show()

In [ ]:
west_bengal_data['detrended_monthly_seasonality'] = west_bengal_data['detrended_daily_seasonality'] - monthly_seasonality

In [ ]:
forecast=west_bengal_data['linear_trend'][-24*486:]+daily_seasonality[-24*486:]+monthly_seasonality[-24*486:]
actual=west_bengal_data['Hourly Demand Met (in MW)'][-24*486:]
mse=mean_squared_error(actual,forecast)
mae=mean_absolute_error(actual,forecast)
print(mse,mae)

In [ ]:
west_bengal_data['is_weekend'] = west_bengal_data['DateTime'].dt.dayofweek >= 5  # Saturday and Sunday are considered weekends

In [ ]:
west_bengal_data.sample(10)

In [ ]:
weekend_dummies=pd.get_dummies(west_bengal_data['is_weekend'], drop_first=True)

In [ ]:
lr_model_weekly = LinearRegression()
lr_model_weekly.fit(weekend_dummies[:-24*486], west_bengal_data['detrended_monthly_seasonality'][:-24*486])

# Predict seasonality from 
weekly_seasonality= lr_model_weekly.predict(weekend_dummies)

In [ ]:
lr_model_weekly.intercept_

In [ ]:
lr_model_weekly.coef_

In [ ]:
west_bengal_data['residuals'] = west_bengal_data['detrended_monthly_seasonality'] - weekly_seasonality

In [ ]:
forecast=west_bengal_data['linear_trend'][-24*486:]+daily_seasonality[-24*486:]+monthly_seasonality[-24*486:]+weekly_seasonality[-24*486:]
actual=west_bengal_data['Hourly Demand Met (in MW)'][-24*486:]
mse=mean_squared_error(actual,forecast)
mae=mean_absolute_error(actual,forecast)
print(mse,mae)

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=actual, name='Observed'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=forecast, name='Forecast_Dummy',mode='lines',opacity=0.6))  # 60% opaque (40% transparent)
fig.update_layout(
    title='Final Forecast vs Observed using LR model',
    xaxis_title='DateTime',
    yaxis_title='Hourly Demand Met (in MW)',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'], y=west_bengal_data['residuals'], name='Observed'))
fig.update_layout(
    title='Residuals',
    xaxis_title='DateTime',
    yaxis_title='Residuals',
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="date"  # Use date scale for x-axis
    # ),
    template='plotly_white'
)
fig.show()

In [ ]:

from statsmodels.tsa.stattools import acf,pacf
import numpy as np
residuals = west_bengal_data['residuals']

# Compute ACF and confidence intervals
acf_values = acf(residuals, nlags=600, fft=True)  # nlags = max lag to compute
conf_int = 1.96 / np.sqrt(len(residuals))  # 95% confidence interval

# Create Plotly figure
fig = go.Figure()

# Add ACF bars
fig.add_trace(
    go.Bar(
        x=list(range(len(acf_values))),
        y=acf_values,
        name="ACF",
    )
)

# Add confidence interval lines
fig.add_hline(
    y=conf_int,
    line_dash="dot",
    line_color="red",
    annotation_text="95% CI",
    annotation_position="bottom right"
)
fig.add_hline(
    y=-conf_int,
    line_dash="dot",
    line_color="red"
)

# Customize layout
fig.update_layout(
    title="Autocorrelation (ACF) Plot",
    xaxis_title="Lag",
    yaxis_title="Correlation",
    showlegend=False,
    template="plotly_white",
    # xaxis=dict(
    #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
    #     type="linear"  # Use linear scale for x-axis
    # )
)

fig.show()

In [ ]:
# 1. Compute PACF values (nlags = number of lags to compute)
pacf_values = pacf(west_bengal_data['residuals'], nlags=400)

# 2. Plot using Plotly
lags = list(range(len(pacf_values)))

fig = go.Figure()

# Bars
fig.add_trace(go.Bar(x=lags, y=pacf_values, name='PACF'))

# Confidence intervals (approx ±1.96/√n)
import numpy as np
conf_level = 1.96 / np.sqrt(len(west_bengal_data['residuals']))

fig.add_shape(type="line", x0=0, x1=len(lags), y0=conf_level, y1=conf_level,
              line=dict(color="red", dash="dash"))
fig.add_shape(type="line", x0=0, x1=len(lags), y0=-conf_level, y1=-conf_level,
              line=dict(color="red", dash="dash"))

# Layout
fig.update_layout(title="Partial Autocorrelation Function (PACF)",
                  xaxis_title="Lag",
                  yaxis_title="PACF",
                  showlegend=False)
fig.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(west_bengal_data['residuals'], autolag='AIC')
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Used lags:", result[2])
print("Number of observations:", result[3])
print("Critical Values:")
for k, v in result[4].items():
    print(f"  {k}%: {v}")

In [ ]:
# tbats_model_fitted=model_daily.fit(west_bengal_data['residuals'][:-24*486])

In [ ]:
# residuals_pattern = tbats_model_fitted.forecast(steps=forecast_steps)

In [ ]:
# fig= go.Figure()
# fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['residuals'], name='Observed'))
# fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=residuals_pattern, name='Forecast_Dummy',mode='lines',opacity=0.6))  # 60% opaque (40% transparent)
# fig.update_layout(
#     title='Residual Modelling Using TBATs',
#     xaxis_title='DateTime',
#     yaxis_title='Residuals',
#     # xaxis=dict(
#     #     rangeslider=dict(visible=True),  # Enable range slider for the bottom subplot
#     #     type="date"  # Use date scale for x-axis
#     # ),
#     template='plotly_white'
# )
# fig.show()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
# SARIMA(p,d,q)(P,D,Q,24) where 24 is the seasonal period
model = SARIMAX(west_bengal_data['residuals'][:-24*486], order=(1, 0, 1), seasonal_order=(1, 1, 1, 24))
results = model.fit()
print(results.summary())

In [ ]:
train_pred = results.get_prediction(start=0, end=len(west_bengal_data['residuals'][:-24*486])-1)
train_mean = train_pred.predicted_mean

In [ ]:
type(train_mean)

In [ ]:
forecast = results.get_forecast(steps=forecast_steps)
test_mean = forecast.predicted_mean
test_ci = forecast.conf_int()

In [ ]:
combined = pd.concat([train_mean, test_mean], axis=0)

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=west_bengal_data['residuals'][-forecast_steps:], name='Observed'))
fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=test_mean, name='Forecast', mode='lines', opacity=0.6))  # 60% opaque (40% transparent)
# fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=test_ci.iloc[:, 0], fill=None, mode='lines', line=dict(width=0)))
# fig.add_trace(go.Scatter(x=west_bengal_data['DateTime'][-forecast_steps:], y=test_ci.iloc[:, 1], fill='tonexty', mode='lines', line=dict(width=0),opacity=0.2,  # 20% opaque (80% transparent) 
#                          name='95% CI'))
fig.update_layout(title='SARIMAX: Actual vs Predicted', template='plotly_white')
fig.show()

In [ ]:
west_bengal_data['sarimax_residuals'] = west_bengal_data['residuals'] - combined

In [ ]:
west_bengal_data.info()

In [ ]:
len(daily_seasonality[-24*486:])

In [ ]:
forecast=west_bengal_data['linear_trend'][-24*486:]+daily_seasonality[-24*486:]+monthly_seasonality[-24*486:]+weekly_seasonality[-24*486:] +test_mean
actual=west_bengal_data['Hourly Demand Met (in MW)'][-24*486:]
mse=mean_squared_error(actual,forecast)
mae=mean_absolute_error(actual,forecast)
print(mse,mae)